In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import datetime
import seaborn as sns
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

In [2]:
# Note, If this file exceeds your RAM, add code "del dfX" after the old dataframe is no longer in use (ex: del df1)

In [3]:
# Read in yearly flight data
# 2017.csv refers to flight data from 2017. Find and replace 2017 to use other years
df1 = pd.read_csv('2017.csv', index_col=None)
df1.head(3).append(df1.tail(3))
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [4]:
# Drop features that are not needed for the model
df1 = df1.drop(columns=['OP_CARRIER_FL_NUM','DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

In [5]:
# Drop non-top 4 airlines.
df2 = df1[df1['OP_CARRIER'].isin(['UA','DL','WN','AA'])]

In [6]:
df2.info()
# print(df1['ORIGIN'].nunique())
print(df2['OP_CARRIER'].nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3733833 entries, 0 to 5674620
Data columns (total 10 columns):
FL_DATE         object
OP_CARRIER      object
ORIGIN          object
DEST            object
CRS_DEP_TIME    int64
DEP_DELAY       float64
CRS_ARR_TIME    int64
ARR_DELAY       float64
AIR_TIME        float64
DISTANCE        float64
dtypes: float64(4), int64(2), object(4)
memory usage: 313.4+ MB
4


In [7]:
# Busiest airports in the US
# "ATL","DFW","DEN","ORD","LAX","JFK","LAS","MCO","MIA","CLT","SEA","PHX","EWR","SFO","IAH","BOS","FLL","MSP","LGA","DTW","PHL","SLC","DCA","SAN","BWI","TPA","AUS","IAD","BNA","MDW"
df3 = df2[df2['ORIGIN'].isin(["ATL","DFW","DEN","ORD","LAX","JFK","LAS","MCO","MIA","CLT","SEA","PHX","EWR","SFO","IAH","BOS","FLL","MSP","LGA","DTW","PHL","SLC","DCA","SAN","BWI","TPA","AUS","IAD","BNA","MDW"])]
df4 = df3[df3['DEST'].isin(["ATL","DFW","DEN","ORD","LAX","JFK","LAS","MCO","MIA","CLT","SEA","PHX","EWR","SFO","IAH","BOS","FLL","MSP","LGA","DTW","PHL","SLC","DCA","SAN","BWI","TPA","AUS","IAD","BNA","MDW"])]

In [8]:
df4.info()
print(df4['ORIGIN'].nunique())
print(df4['OP_CARRIER'].nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849470 entries, 0 to 5674620
Data columns (total 10 columns):
FL_DATE         object
OP_CARRIER      object
ORIGIN          object
DEST            object
CRS_DEP_TIME    int64
DEP_DELAY       float64
CRS_ARR_TIME    int64
ARR_DELAY       float64
AIR_TIME        float64
DISTANCE        float64
dtypes: float64(4), int64(2), object(4)
memory usage: 155.2+ MB
30
4


In [9]:
# Add Weather
# df2 = pd.read_csv('Weather2017.csv', index_col=0)
dfWea = pd.read_csv('Weather2017.csv', index_col=None)
dfWea.head(3).append(dfWea.tail(3))

,LST_DATE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas
0,20170101,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0
1,20170102,-1.7,6.0,-7.6,0.0,-1.4,0.0,-0.3,0.0,10.9,35.5,10.3,0.0,16.1,8.7
2,20170103,-7.0,1.4,-10.3,7.0,2.8,13.1,-1.3,0.0,15.0,2.8,10.8,0.0,12.2,0.0
362,20171229,-15.0,0.8,1.0,0.0,-14.2,0.0,8.3,88.2,4.8,0.0,13.8,0.0,6.6,0.0
363,20171230,-20.9,0.0,-3.3,0.0,-13.0,1.1,4.1,7.0,4.3,0.0,10.1,0.0,8.4,0.0
364,20171231,-20.7,0.0,-6.8,2.5,-17.0,0.0,3.2,0.0,0.4,0.0,11.9,0.0,-1.6,0.0


In [10]:
# Convert date formats to the dame to merge dataframes
dfWea['LST_DATE'] =  pd.to_datetime(dfWea['LST_DATE'], format='%Y%m%d')

In [11]:
df4['FL_DATE'] =  pd.to_datetime(df4['FL_DATE'], format='%Y-%m-%d')
print(df1.iloc[:, 0])

0          2017-01-01
1          2017-01-01
2          2017-01-01
3          2017-01-01
4          2017-01-01
              ...    
5674616    2017-12-31
5674617    2017-12-31
5674618    2017-12-31
5674619    2017-12-31
5674620    2017-12-31
Name: FL_DATE, Length: 5674621, dtype: object


In [12]:
# Merge flight and weather dataframes
df5 = pd.merge(df4,dfWea,left_on='FL_DATE',right_on='LST_DATE',how='right')

In [13]:
df5['DAY'] = pd.DatetimeIndex(df5['FL_DATE']).day
df5['MONTH'] = pd.DatetimeIndex(df5['FL_DATE']).month

In [14]:
# Get holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
dr = pd.date_range(start='2017-01-01', end='2017-12-31')
holidays = cal.holidays(start=dr.min(), end=dr.max())
df5['HOLIDAY'] = df5['FL_DATE'].isin(holidays)
df5['HOLIDAY'] = df5['HOLIDAY'].astype(int)

In [15]:
# Convert amount delayed to binary
status = []

for value in df5['ARR_DELAY']:
    if value < 0:
        status.append(0)
    else:
        status.append(1)
df5['FLIGHT_STATUS'] = status
df5.head(2)


statusdepart = []

for value in df5['DEP_DELAY']:
    if value < 0:
        statusdepart.append(0)
    else:
        statusdepart.append(1)
df5['INCOMING_Delay'] = statusdepart
df5.head(2)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,LST_DATE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas,DAY,MONTH,HOLIDAY,FLIGHT_STATUS,INCOMING_Delay
0,2017-01-01,AA,JFK,LAX,800,31.0,1142,27.0,347.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,1,1
1,2017-01-01,AA,LAX,JFK,900,34.0,1727,42.0,289.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,1,1


In [16]:
# Dummy encode airline, origin, and destination
# Note: There was a tight timeline in the data science fellowship. I thought investing my time in increasing the number of years
# in the dataset and optimizing the models would be the best use of my time. However, accuarcy improvements were small. If you are
# trying to recreate this project, explore other options outside of dummy encoding these variables.
df5 = pd.get_dummies(data=df5, columns=['OP_CARRIER','ORIGIN','DEST'])

In [17]:
df5.head()

,FL_DATE,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,LST_DATE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas,DAY,MONTH,HOLIDAY,FLIGHT_STATUS,INCOMING_Delay,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_AUS,ORIGIN_BNA,ORIGIN_BOS,ORIGIN_BWI,ORIGIN_CLT,ORIGIN_DCA,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_DTW,ORIGIN_EWR,ORIGIN_FLL,ORIGIN_IAD,ORIGIN_IAH,ORIGIN_JFK,ORIGIN_LAS,ORIGIN_LAX,ORIGIN_LGA,ORIGIN_MCO,ORIGIN_MDW,ORIGIN_MIA,ORIGIN_MSP,ORIGIN_ORD,ORIGIN_PHL,ORIGIN_PHX,ORIGIN_SAN,ORIGIN_SEA,ORIGIN_SFO,ORIGIN_SLC,ORIGIN_TPA,DEST_ATL,DEST_AUS,DEST_BNA,DEST_BOS,DEST_BWI,DEST_CLT,DEST_DCA,DEST_DEN,DEST_DFW,DEST_DTW,DEST_EWR,DEST_FLL,DEST_IAD,DEST_IAH,DEST_JFK,DEST_LAS,DEST_LAX,DEST_LGA,DEST_MCO,DEST_MDW,DEST_MIA,DEST_MSP,DEST_ORD,DEST_PHL,DEST_PHX,DEST_SAN,DEST_SEA,DEST_SFO,DEST_SLC,DEST_TPA
0,2017-01-01,800,31.0,1142,27.0,347.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-01-01,900,34.0,1727,42.0,289.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2017-01-01,1130,51.0,1958,42.0,284.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2017-01-01,700,-4.0,1045,-22.0,362.0,2586.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2017-01-01,2100,0.0,527,-30.0,272.0,2475.0,2017-01-01,-3.9,0.0,-6.0,0.0,0.6,0.0,-0.3,0.0,6.8,17.4,9.9,0.0,15.3,0.0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# df5.info()

In [19]:
df5 = df5.drop(columns=['FL_DATE','LST_DATE','DEP_DELAY'])

In [20]:
df5.to_csv('All2017Flight_Weather.csv')

In [21]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849470 entries, 0 to 1849469
Data columns (total 88 columns):
CRS_DEP_TIME              int64
CRS_ARR_TIME              int64
ARR_DELAY                 float64
AIR_TIME                  float64
DISTANCE                  float64
T_DAILY_MEAN_Midwest      float64
P_DAILY_CALC_Midwest      float64
T_DAILY_MEAN_Mountain     float64
P_DAILY_CALC_Mountain     float64
T_DAILY_MEAN_Northeast    float64
P_DAILY_CALC_Northeast    float64
T_DAILY_MEAN_PNW          float64
P_DAILY_CALC_PNW          float64
T_DAILY_MEAN_Southeast    float64
P_DAILY_CALC_Southeast    float64
T_DAILY_MEAN_Southwest    float64
P_DAILY_CALC_Southwest    float64
T_DAILY_MEAN_Texas        float64
P_DAILY_CALC_Texas        float64
DAY                       int64
MONTH                     int64
HOLIDAY                   int64
FLIGHT_STATUS             int64
INCOMING_Delay            int64
OP_CARRIER_AA             uint8
OP_CARRIER_DL             uint8
OP_CARRIER_UA        